In [1]:
# import libraries
from gettext import npgettext
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
from statistics import mean, stdev

import json

In [2]:
# setup spark
spark = (
    SparkSession.builder.appName("aggregate data for first 3 final model variables")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "15g")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/29 12:57:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/09/29 12:57:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/09/29 12:57:41 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
postcode_df = spark.read.csv("../data/curated/Grouping.csv", header = "true")
persona_df = spark.read.csv("../data/curated/abs_cleaned.csv",  header = "true")

In [4]:
persona_df = persona_df.withColumnRenamed("_c0","postcode")

In [5]:
# mean age
cond = ["postcode"]
total_df = postcode_df.join(persona_df,cond, "left")

In [6]:
total_df

22/09/29 12:57:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/09/29 12:57:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, Managers_%, Professionals_%, Technicians and Trades Workers_%, Community and Personal Service Workers_%, Clerical and Administrative Workers_%, Sales Workers_%, Machinery Operators and Drivers_%, Labourers_%, Total
 Schema: _c0, mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and th

postcode,merchant_abn,Fortnight,count_postcode_merchant,total_order_per_fornight,proportion_of_postcode,mean_Age,min_Age,q1_Age,median_Age,q3_Age,max_Age,Millen_Z%,Oceania and Antarctica,North-West Europe,Southern and Eastern Europe,North Africa and the Middle East,South-East Asia,North-East Asia,Southern and Central Asia,Americas,Sub-Saharan Africa,mean_Salary,median_Salary,min_Salary,q1_Salary,q3_Salary,max_Salary,Managers_%,Professionals_%,Technicians and Trades Workers_%,Community and Personal Service Workers_%,Clerical and Administrative Workers_%,Sales Workers_%,Machinery Operators and Drivers_%,Labourers_%,Total
4479,10530696903,5,1,5,0.2,51.0,46,46,46,46,46,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,301.67,75.0,0.0,0.0,543.75,575.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15
7468,10530696903,5,1,5,0.2,46.34788937409025,0,24,43,60,87,0.27510917030567683,0.9453125,0.040625,0.0,0.0,0.0078125,0.0,0.0,0.00625,0.0,602.51,575.0,225.0,350.0,1125.0,1850.0,0.0988700564971751,0.03954802259887,0.057909604519774,0.0381355932203389,0.0437853107344632,0.0409604519774011,0.03954802259887,0.0932203389830508,708
2178,10530696903,5,1,5,0.2,43.982105015304924,0,19,39,57,96,0.31057216858959263,0.7152649617716846,0.024782494068020036,0.12760348009491168,0.06011073029264435,0.024782494068020036,0.028209860268916424,0.006063801740047456,0.011863959926179805,0.001318217769575534,587.8,725.0,0.0,350.0,1125.0,1625.0,0.0883325460557392,0.0555030703826169,0.0798299480396787,0.0323571091166745,0.0772319319792158,0.0387340576287198,0.0415682569674067,0.0425129900803023,4234
6210,10530696903,5,1,5,0.2,47.947262108637425,0,21,44,64,100,0.25353122850158816,0.7490059955822026,0.16307983591038183,0.009608709372041653,0.003739349952666...,0.023856106027137898,0.005758914484064374,0.012290943515304513,0.007920479646576207,0.024739665509624486,626.51,575.0,0.0,350.0,1125.0,1875.0,0.0342253191240192,0.0536216184564937,0.0737205761798805,0.0463022602178241,0.042979271577468,0.040300386462115,0.0385291017683569,0.0431988523246281,68312
4573,10530696903,5,1,5,0.2,44.45165583057678,0,17,41,58,101,0.24124558001596896,0.8349875423763428,0.10529755340440305,0.009230894906669933,0.002287301392803...,0.008332312216640117,0.003267573418290242,0.003512641424662...,0.017073071110566516,0.016011109749622186,620.34,725.0,75.0,350.0,1125.0,1875.0,0.0526716395189526,0.1040873801187395,0.0724996194245699,0.0554117826153143,0.0535469630080682,0.0481808494443598,0.0170497792662505,0.0398081899832546,26276
6701,10739268011,5,1,1,1.0,43.4642921388739,0,19,39,56,96,0.27594891167476165,0.8164198606271778,0.05945121951219512,0.033101045296167246,0.001524390243902439,0.048344947735191636,0.01524390243902439,0.007186411149825784,0.006315331010452962,0.012412891986062718,564.57,725.0,75.0,350.0,1125.0,1875.0,0.0840093744366324,0.0531819001261943,0.0557057869118442,0.0438074634937804,0.0531819001261943,0.0299260861727059,0.0290246980349738,0.0647196682891653,5547
3959,10901349044,5,1,20,0.05,49.24027657735523,0,22,48,65,94,0.1815038893690579,0.8957934990439771,0.06883365200764818,0.01338432122370937,0.003824091778202...,0.002868068833652...,0.0,0.004780114722753346,0.005736137667304...,0.004780114722753346,515.65,575.0,75.0,350.0,900.0,1375.0,0.1326973113616652,0.0650477016478751,0.044232437120555,0.03122289679098,0.044232437120555,0.0173460537727666,0.0268863833477883,0.044232437120555,1153
4426,10901349044,5,1,20,0.05,54.22105263157895,4,41,51,63,84,0.11578947368421053,0.9583333333333334,0.041666666666666664,0.0,0.0,0.0,0.0,0.0,0.0,0.0,594.25,575.0,0.0,225.0,900.0,1625.0,0.2142857142857142,0.0317460317460317,0.0396825396825396,0.0317460317460317,0.0873015873015873,0.0,0.0238095238095238,0.0793650793650793,126
6369,10901349044,5,1,20,0.05,49.06018518518518,0,25,46,64,90,0.22530864197530864,0.8803278688524591,0.06885245901639345,0.00819672131147541,0.0,0.019672131147540985,0.0,0.00819672131147541,0.0,0.014754098360655738,698.09,725.0,105.0,350.0,1375.0,1875.0,0.1664167916041979,0.0434782608695652,

In [7]:
listofmult = total_df.columns[6:]
listofmult

['mean_Age',
 'min_Age',
 'q1_Age',
 'median_Age',
 'q3_Age',
 'max_Age',
 'Millen_Z%',
 'Oceania and Antarctica',
 'North-West Europe',
 'Southern and Eastern Europe',
 'North Africa and the Middle East',
 'South-East Asia',
 'North-East Asia',
 'Southern and Central Asia',
 'Americas',
 'Sub-Saharan Africa',
 'mean_Salary',
 'median_Salary',
 'min_Salary',
 'q1_Salary',
 'q3_Salary',
 'max_Salary',
 'Managers_%',
 'Professionals_%',
 'Technicians and Trades Workers_%',
 'Community and Personal Service Workers_%',
 'Clerical and Administrative Workers_%',
 'Sales Workers_%',
 'Machinery Operators and Drivers_%',
 'Labourers_%',
 'Total']

In [8]:
total_df = total_df.withColumn("arr", F.struct(*[(F.col(x)*F.col('proportion_of_postcode')).alias(x) for x in listofmult]))\
  .select("postcode","merchant_abn","Fortnight","count_postcode_merchant","total_order_per_fornight","arr.*")

In [9]:
exprs = {x: "sum" for x in listofmult}
final_df1 = total_df.groupBy("merchant_abn","Fortnight").agg(exprs)

In [10]:
final_df1.toPandas().to_csv('../data/curated/Postcode_Persona.csv', index = False)

22/09/29 12:57:45 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, Managers_%, Professionals_%, Technicians and Trades Workers_%, Community and Personal Service Workers_%, Clerical and Administrative Workers_%, Sales Workers_%, Machinery Operators and Drivers_%, Labourers_%, Total
 Schema: _c0, mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, 

In [11]:
final_df1

22/09/29 12:58:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, Managers_%, Professionals_%, Technicians and Trades Workers_%, Community and Personal Service Workers_%, Clerical and Administrative Workers_%, Sales Workers_%, Machinery Operators and Drivers_%, Labourers_%, Total
 Schema: _c0, mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, 

22/09/29 12:59:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, Managers_%, Professionals_%, Technicians and Trades Workers_%, Community and Personal Service Workers_%, Clerical and Administrative Workers_%, Sales Workers_%, Machinery Operators and Drivers_%, Labourers_%, Total
 Schema: _c0, mean_Age, min_Age, q1_Age, median_Age, q3_Age, max_Age, Millen_Z%, Oceania and Antarctica, North-West Europe, Southern and Eastern Europe, North Africa and the Middle East, South-East Asia, North-East Asia, Southern and Central Asia, Americas, Sub-Saharan Africa, mean_Salary, median_Salary, min_Salary, q1_Salary, q3_Salary, max_Salary, 

merchant_abn,Fortnight,sum(Professionals_%),sum(median_Age),sum(q1_Salary),sum(Machinery Operators and Drivers_%),sum(Southern and Central Asia),sum(max_Age),sum(min_Salary),sum(Oceania and Antarctica),sum(North-East Asia),sum(Sales Workers_%),sum(mean_Salary),sum(Sub-Saharan Africa),sum(Technicians and Trades Workers_%),sum(Community and Personal Service Workers_%),sum(North Africa and the Middle East),sum(q3_Age),sum(Total),sum(North-West Europe),sum(max_Salary),sum(mean_Age),sum(q1_Age),sum(Labourers_%),sum(Southern and Eastern Europe),sum(min_Age),sum(Managers_%),sum(South-East Asia),sum(Americas),sum(median_Salary),sum(q3_Salary),sum(Clerical and Administrative Workers_%),sum(Millen_Z%)
50131473008,5,0.042941631427480226,41.666666666666664,308.3333333333333,0.042029279687339195,0.03128522581457509,96.99999999999999,50.0,0.7888710740349993,0.007352445644589651,0.04425395237983107,470.8333333333333,0.005264235064190...,0.053774499038758125,0.03665982125107133,0.03269693320027259,59.666666666666664,10679.0,0.029099705071489613,1375.0,45.997441620897206,19.666666666666664,0.06500882205073968,0.07321245100137719,0.0,0.0580910116088286,0.028392386423014226,0.003825543745491...,491.66666666666663,900.0,0.041161756198535265,0.270331900589163
73221370757,5,0.11336037037901708,43.5,357.5,0.02071768561958178,0.01590248135079528,93.3,75.0,0.7989929412782624,0.0227466843858208,0.03811852064917305,670.328,0.008358878123939296,0.06262631252794969,0.04662092362548657,0.004950188083383...,58.89999999999999,7619.299999999999,0.08493369134881053,1862.5,46.46060102926346,22.900000000000002,0.04654049517049972,0.01840036143037203,0.0,0.07502698334595555,0.03285332774538342,0.01286144625323287,697.5,1255.0,0.05374588804383884,0.25678929782229176
96906278264,5,0.07016346495824526,45.50000000000001,345.5729166666667,0.031249909097184166,0.008719711302812376,91.41666666666666,108.125,0.8820419493932198,0.006071275713920277,0.032462049282354674,596.2916666666669,0.005304878676785956,0.04949217990952279,0.039936434901441276,0.003260686710461...,61.66666666666666,7691.5,0.05968386848757546,1710.4166666666663,48.69000412997744,25.000000000000004,0.055792102554133305,0.01776403507386255,2.8333333333333335,0.13000751361674187,0.01159787996476095,0.005555714676601109,597.9166666666666,1073.1770833333333,0.0422015507348211,0.2247300804592805
84057522285,6,0.08231809651771568,41.709090909090904,349.0909090909093,0.032751336065330115,0.013532119771650268,90.79999999999995,72.90909090909093,0.8190257478105476,0.017638163362477637,0.032625961170383765,634.798909090909,0.010000597725332723,0.0607740017242745,0.04451492709549662,0.008288912514595523,57.87272727272727,8539.509090909089,0.07068655359098372,1962.090909090909,45.328463811151536,21.072727272727267,0.05153312886166294,0.022697677806059417,1.9272727272727268,0.09179992478993608,0.029409571498368432,0.008720655919984686,680.9090909090908,1221.0227272727277,0.05045625047876569,0.2684448567728932
49152510995,7,0.09799219932291714,40.33333333333333,341.66666666666663,0.02352505688749553,0.01528662135710349,92.0,50.0,0.7983631415475045,0.014170587281195133,0.03548827165294344,641.7833333333333,0.009049718986196937,0.05721194023017796,0.04503681176174456,0.010116896289949022,57.33333333333333,6748.0,0.05124221179894242,1958.3333333333333,44.87631732609827,21.666666666666664,0.0374565575525787,0.04305902856232744,0.0,0.0700079828228263,0.045124033590729966,0.013587760586051028,691.6666666666666,1300.0,0.05629994517329946,0.31089567806899254
51016899219,7,0.08123825568386216,41.75454545454544,373.18181818181847,0.03038014960206793,0.015054445769764389,89.78181818181821,98.77272727272742,0.8427148962928206,0.018867299309267123,0.03313240981783917,630.4541818181816,0.008897092691085708,0.05967430235424205,0.043153664826130374,0.00620696025986196,57.31818181818181,7690.481818181822,0.06110989922140846,1864.8863636363628,45.29402822544746,21.990909090909092,0.05805530933936261,0.01694386030986725,1.5181818181818